In [8]:
#!/usr/bin/env python3
import os
import numpy as np
import SimpleITK as sitk
from PIL import Image
import glob
import re

# ——— CONFIG ———
BASE_DIR = r"C:\Users\aruna\Documents\alz\dataset d"
OUTPUT_BASE_DIR = "extracted_slices_sitk"
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

# Pattern to match subject IDs from OAS1_0001_MR1 to OAS1_0042_MR1
pattern = r"OAS1_00(\d{1,2})_MR1"

# Function to process a single MRI file
def process_mri(subject_id):
    print(f"\n===== Processing {subject_id} =====")
    
    # Construct the path to the MRI file
    img_path = os.path.join(
        BASE_DIR, 
        subject_id, 
        "PROCESSED", 
        "MPRAGE", 
        "T88_111", 
        f"{subject_id}_mpr_n4_anon_111_t88_masked_gfc.img"
    )
    
    # Create subject-specific output directory
    output_dir = os.path.join(OUTPUT_BASE_DIR, subject_id)
    os.makedirs(output_dir, exist_ok=True)
    
    # ——— SANITY CHECK ———
    if not os.path.isfile(img_path):
        print(f"⚠️ Couldn't find IMG file: {img_path!r}. Skipping.")
        return False
    
    try:
        # ——— READ WITH SimpleITK ———
        image = sitk.ReadImage(img_path)
        size = image.GetSize()  # (X, Y, Z)
        dtype = sitk.GetArrayFromImage(image).dtype
        print(f"🔍 Loaded image.  Size (X×Y×Z): {size},  NumPy dtype: {dtype}")
        
        # ——— GET NUMPY ARRAY ———
        # Note: GetArrayFromImage returns an array of shape (Z, Y, X)
        data = sitk.GetArrayFromImage(image)
        print(f"🔍 NumPy array shape (Z×Y×X): {data.shape}")
        
        if data.ndim != 3:
            print(f"⚠️ Expected a 3-D volume, but got array with shape {data.shape}. Skipping.")
            return False
        
        # ——— NORMALIZE INTO 8-BIT ———
        data = data.astype(np.float32)
        mn, mx = np.nanmin(data), np.nanmax(data)
        print(f"🔍 Data range before scaling: [{mn:.3f}, {mx:.3f}]")
        if mx > mn:
            data = (data - mn) / (mx - mn) * 255.0
        data = np.nan_to_num(data).astype(np.uint8)
        
        # ——— SLICE & SAVE ———
        for z in range(data.shape[0]):
            slice2d = data[z, :, :]  # (Y, X)
            out_fn = os.path.join(output_dir, f"slice_{z:03d}.png")
            Image.fromarray(slice2d).save(out_fn)
        
        print(f"✅ Done! {data.shape[0]} slices written to '{output_dir}/'")
        return True
        
    except Exception as e:
        print(f"❌ Error processing {subject_id}: {str(e)}")
        return False

# Main execution
def main():
    success_count = 0
    total_count = 0
    
    # Process subjects from OAS1_0001_MR1 to OAS1_0042_MR1
    for i in range(1, 43):
        subject_id = f"OAS1_{i:04d}_MR1"
        total_count += 1
        if process_mri(subject_id):
            success_count += 1
    
    print(f"\n===== Summary =====")
    print(f"Successfully processed {success_count} out of {total_count} subjects")
    print(f"Results saved to '{os.path.abspath(OUTPUT_BASE_DIR)}'")

if __name__ == "__main__":
    main()


===== Processing OAS1_0001_MR1 =====
🔍 Loaded image.  Size (X×Y×Z): (176, 208, 176),  NumPy dtype: int16
🔍 NumPy array shape (Z×Y×X): (176, 208, 176)
🔍 Data range before scaling: [0.000, 3126.000]
✅ Done! 176 slices written to 'extracted_slices_sitk\OAS1_0001_MR1/'

===== Processing OAS1_0002_MR1 =====
🔍 Loaded image.  Size (X×Y×Z): (176, 208, 176),  NumPy dtype: int16
🔍 NumPy array shape (Z×Y×X): (176, 208, 176)
🔍 Data range before scaling: [0.000, 2595.000]
✅ Done! 176 slices written to 'extracted_slices_sitk\OAS1_0002_MR1/'

===== Processing OAS1_0003_MR1 =====
🔍 Loaded image.  Size (X×Y×Z): (176, 208, 176),  NumPy dtype: int16
🔍 NumPy array shape (Z×Y×X): (176, 208, 176)
🔍 Data range before scaling: [0.000, 3149.000]
✅ Done! 176 slices written to 'extracted_slices_sitk\OAS1_0003_MR1/'

===== Processing OAS1_0004_MR1 =====
🔍 Loaded image.  Size (X×Y×Z): (176, 208, 176),  NumPy dtype: int16
🔍 NumPy array shape (Z×Y×X): (176, 208, 176)
🔍 Data range before scaling: [0.000, 3216.000]
✅